# Yubico Keys
---

[Yubico](http://www.yubico.com) manufactures low-cost security key fobs for multi/two-factor authentication (2FA), One-Time Passwords (OTP) and Univeral Second Factor (U2F). By requiring a physical device with a key that quickly expires in order to authenticate the user, cyber attacks become more difficult to pull off. Using the Yubikey 5 NFC, shown below, python processes and applications can be secured. 

<img src="jupyter-figures/Yubikey-5NFC.jpg" style="max-height: 500px; max-width: 500px;"/>

In this development experiment, a user action will be authenticated using both a user password as well as a Yubikey 5 NFC authenticator. This experiment will hopefully pave the way for future work in 2FA with the Yubico product line.

In addition to the Yubico API, authentication methods are provided for encrypting files and checking passwords against a generated hash. These methods allow for additional layers of security to be added to future Python projects and APIs.

---

## <a name="TOC"></a> Table of Contents
1. [Secure Local Storage of Sensitive Files](#secure-local-storage)
2. [Secure Password Verification](#secure-password-verification)
3. [Validate OTP](#validate-otp)



In [1]:
# -------------------- CONFIGURE ENVIRONMENT -------------------- #

%reset -f

from passlib.context import CryptContext
from cryptography.fernet import Fernet
from getpass import getpass
from pathlib import Path
import requests
import random
import string
import json
import sys
import os

path_keys = Path("keys")



## <a name="secure-local-storage"></a> [Secure Local Storage of Sensitive Files](#TOC)

One topic which has bothered me for some time now is how to safely and reliably store credentials and keys in a python software ecosystem. With a compiled language, the keys can be stored in a library of string literals and compiled into the executable program where it would have to be reliably decompiled for a consumer to obtain elevated access to the system. This challenge is by no means small but it is not impossible either. One alternative is to store keys in a database but then you have to somehow grant the program access to the database which repeats the issue over again. Whitelisting the IP address of your software is useful but it's too broad a security policy.

In order to connect to Yubico products, client identifiers and keys must be passed from this software to the Yubico API. For this reason, I will begin by safely storing this information so that my JupyterLab notebook can safely demonstrate how the Yubico validation process works in Python without exposing my sensitive information to the reader.

The process below details how to use [Fernet symmetric encryption](https://cryptography.io/en/latest/fernet/) to encrypt and decrypt a local file using a securely generated keyfile. Ideally, configuration files storing sensitive account information can be safely exchanged over insecure mediums because consumers without the keyfile would be forced to crack the configurations manually. However, this solution does not provide a way to unlock the configuration without exposing the keyfile. In other words, it does not allow the program to run automatically without the administrator providing the keyfile.

According to the [Fernet specifications](https://github.com/fernet/spec/blob/master/Spec.md), this encryption method is built on top of the AES 128-bit protocol. Although this encryption protocol is trusted and widely used (and therefore widely scrutinized), the number of bits for the hash is limited considering that modern cracking techniques can bypass a 128-bit encryption. However, the AES protocol in Fernet is run under the cipher blocker chaining (CBC) mode that makes each subsequent cyphertext block dependent on all previously processed plaintext blocks. By interrelating the block cyphers, this AES protocol becomes significantly more difficult to bypass.



In [2]:
# -------------------- ENCRYPT CONFIGURATION FILE -------------------- #

def generate_keyfile(filename):
    """
    Generates a Fernet key and stores the key in
    the provided txt file with the provided path.
    """
    
    if filename[-4:] == ".key":
        if not os.path.isfile(filename):
            os.mknod(filename)
        
        key = Fernet.generate_key()
        with open(filename, "wb+") as file:
            file.write(key)
            file.close()
    else:
        raise TypeError("The provided file must be a key file.")


def load_keyfile(filename):
    """
    Reads the key stored in the provided key file.
    """
    
    if filename[-4:] == ".key":
        if not os.path.isfile(filename):
            os.mknod(filename)
        
        with open(filename, "rb+") as file:
            key = file.read()
            file.close()
        return key
    else:
        raise TypeError("The provided file must be a key file.")


def encrypt_file(source, destination, key):
    """
    Encrypts the provided source file using the
    provided key and saves the encrypted data to
    the destination file.
    """
    
    if not os.path.isfile(source):
        os.mknod(source)
        
    with open(source, "rb+") as file:
        source = file.read()
        file.close()
        
    fernet = Fernet(key)
    encrypted = fernet.encrypt(source)
    
    if not os.path.isfile(destination):
        os.mknod(destination)
    
    with open(destination, "rb+") as file:
        file.write(encrypted)
        file.close()


def decrypt_file(source, destination, key):
    """
    Decrypts the provided source file using the
    provided key and saves the decrypted data to
    the destination file.
    """
    
    if not os.path.isfile(source):
        os.mknod(source)
        
    with open(source, "rb+") as file:
        source = file.read()
        file.close()
        
    fernet = Fernet(key)
    encrypted = fernet.decrypt(source)
    
    if not os.path.isfile(destination):
        os.mknod(destination)
    
    with open(destination, "rb+") as file:
        file.write(encrypted)
        file.close()



In [3]:
# -------------------- TEST ENCRYPTING FILE -------------------- #

# Generate and load keyfile
generate_keyfile("keys/alpha.key")
key = load_keyfile("keys/alpha.key")

# Encrypt configuration file
encrypt_file("config/yubico.json", "config/yubico_secured.json", key)

# Decrypt configuration file
decrypt_file("config/yubico_secured.json", "config/yubico_2.json", key)



In this test, a json file containing sensitive account information for the Yubico API is encrypted using a generated key named `alpha.key`. The resulting secured JSON file is then decrypted and stored in a third file. Within the file browser, users will see that the first file remains untouched, that the secured file cannot be read as the information is encrypted, and that the third file is exactly the same as the first file. Ergo, the JSON file was safely encrypted and decrypted without loss of information.



## <a name="secure-password-verification"></a> [Secure Password Verification](#TOC)

Critically, a secure application requires the ability to facilitate or inhibit specific users on the basis of their ability to verify their authority to interface with the underlying data. Using `passlib`, passwords can be hashed using SHA-256 and later attempts to log into the application can be verified using the locally stored hash. This method allows for a local database of hashes to be stored without keeping sensitive data exposed for attackers.

The goal of this section is to develop the framework for authenticating applications using hashed and salted passwords. These functions will later be deployed in web development contexts.



In [4]:
# -------------------- ENCRYPT PASSWORD -------------------- #

crypt_context = CryptContext(
    schemes=["pbkdf2_sha256"],
    default="pbkdf2_sha256",
    pbkdf2_sha256__default_rounds=30000
)


def encrypt_password(password):
    """
    Encrypts the plaintext password provided.
    """
    
    return crypt_context.hash(password)


def verify_password(password, hashed):
    """
    Verifies if the provided password matches
    the hash for the given account.
    """
    
    return crypt_context.verify(password, hashed)



In [5]:
# -------------------- VERIFY PASSWORD USING HASH -------------------- #

hashed = encrypt_password("Frankenstein")
print(f"Plain: Frankenstein")
print(f"Hash: {hashed}\n")

verified = verify_password("Apple", hashed)
print(f"Verify 'Apple': {verified}")

verified = verify_password("Frankenstein", hashed)
print(f"Verify 'Frankenstein': {verified}")



Plain: Frankenstein
Hash: $pbkdf2-sha256$30000$Y0wJodT6n9MaQ6hVCmGMkQ$/hWmSZQpbc6CT6dTLS0ECNBWW8C/etEqbKTej48oyTs

Verify 'Apple': False
Verify 'Frankenstein': True


This process allows for the hashed passwords to be stored locally and for authorization attempts to be made directly against the hash rather than against a plaintext password. In other words, the hashes could theoretically be exposed without compromising the password itself. This exposure would not be good practice but it does add a layer of security.



## <a name="validate-otp"></a> [Validate OTP](#TOC)

Yubico provides a validation API for verifying OTPs supplied by registered keys. The [documentation](https://developers.yubico.com/yubikey-val/Validation_Protocol_V2.0.html) for the API describes how to format the API call using the `client_id` and `secret_key` provided by Yubico when the product is [registered](https://upgrade.yubico.com/getapikey/) for the OTP-API. For this example, I have provided the bare-minimum amount of parameters to demonstrate this API's effectiveness at validating OTPs.



In [6]:
# -------------------- VERIFY YUBICO OTP -------------------- #

def rand_string(length):
    """
    Generates a random string of numbers and
    letters for crpyographic purposes with the
    character length provided.
    """
    
    nonce = ''.join(random.choice(
        string.ascii_uppercase + string.digits)
                    for _ in range(25))
    
    return nonce


def validate_otp(client_id, otp):
    """
    Validates the provided OTP against the
    Yubico API for the provided client id.
    """
    
    nonce = rand_string(25)

    url = f"https://api.yubico.com/wsapi/2.0/verify?"\
        f"id={client_id}&otp={otp}&nonce={nonce}"

    raw_response = requests.get(url).text.splitlines()

    keys = list(); values = list()
    for field in raw_response:
        if len(field):
            keys.append(field.split("=", 1)[0])
            values.append(field.split("=", 1)[1])

    response = dict(zip(keys, values))
    
    return response



In [7]:
# -------------------- TEST YUBICO OTP -------------------- #

# Load configuration file
file = open("config/yubico.json",) 
config = json.load(file)

# Client settings
client_id = config["client_id"]
# secret_key = config[secret_key]

# OTP for validation
otp = getpass("OTP: ")

# Validate OTP
response = validate_otp(client_id, otp)
status = response["status"]
print(f"Status: {status}")



OTP:  ············································


Status: OK


This test demonstrates how one time passwords can be authenticated using only the Yubico API. Accessing the python process then requires a physical key to provide a valid OTP.



*Written by Austin Dial and Alice Seaborn September 29 - October 03.*